In [ ]:
# Input : BOM HISTORICAL file

# Output: given BOM and GFS features for current time, produces rainfall forecast for the CURRENT hour, although groundtruth is 
# available for current time. The logic is this, we will forecast rainfall on the basis of future values of GFS and BOM
# at timesteps 't+1, t+2 ... t+240' to produce rainfall at timesteps 't+1, t+2 ... t+240'

# Testing: GFS+BOM inputs will be used cross validation. During real-time testing, 
# BOM's variables will become available from our own predictions )

# NOTES: If the groundtruth dataframe contains NaNs due to missing values in BOM Historical file, then these NaN entries will
#        be replaced by zero

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
import scipy.stats as stats
from datetime import datetime, timedelta

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from keras.layers.recurrent import LSTM

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from copy import deepcopy

from keras.models import load_model

%matplotlib inline

In [ ]:
train_ratio = 0.8
df_gfs = pd.read_csv("C:/Shaukat/code/data_rep/gfs/2016_2017/csv/pooley_1hr.csv")
df_bom = pd.read_csv("C:/Shaukat/code/data_rep/sensor_data/houston_rainfall.csv")


In [ ]:
def check_and_fill_null_entries_df_col_by_value(df, col_name, val=0.0):
    # Check if dataframe is null
    is_df_nan = df[col_name].isnull().any()
    if (is_df_nan):
        print 'col %s contains null values. Filling these null values by %f' %(col_name, val)
        df[col_name].fillna(val, inplace=True)

    # Check again if dataframe column is null
    is_df_nan = df[col_name].isnull().any()
    if not is_df_nan:
        print '%s doesnot contain null values. PROCEED' %(col_name)
    
    return df

In [ ]:
def check_and_fill_null_entries_df_col_backwards(df, col_name):
    # Check if dataframe is null
    is_df_nan = df[col_name].isnull().any()
    if (is_df_nan):
        print 'col %s contains null values. Filling these null values' %(col_name)
        df[col_name].fillna(method='bfill', inplace=True)

    # Check again if dataframe column is null
    is_df_nan = df[col_name].isnull().any()
    if not is_df_nan:
        print '%s doesnot contain null values. PROCEED' %(col_name)
    
    return df

In [ ]:
def check_and_fill_null_entries_df(df):
    # Check if dataframe is null
    is_df_nan = df.isnull().any().any()
    if (is_df_nan):
        print 'df contains some null values. Filling these null values'
        df.fillna(method='ffill', inplace=True)

    # Check again if dataframe is null
    is_df_nan = df.isnull().any().any()
    if not is_df_nan:
        print 'passed df doesnot contain null values. PROCEED'
    
    return df

In [ ]:
def check_and_fill_null_entries_df_col(df, col_name):
    # Check if dataframe is null
    is_df_nan = df[col_name].isnull().any()
    if (is_df_nan):
        print 'col %s contains null values. Filling these null values' %(col_name)
        df[col_name].fillna(method='ffill', inplace=True)

    # Check again if dataframe column is null
    is_df_nan = df[col_name].isnull().any()
    if not is_df_nan:
        print '%s doesnot contain null values. PROCEED' %(col_name)
    
    return df

In [ ]:
df_bom.columns

In [ ]:
# Setting up GFS
df_gfs['Unnamed: 0'] = pd.to_datetime(df_gfs['Unnamed: 0'])
df_gfs.set_index('Unnamed: 0', inplace = True)

# Setting up BOM
df_bom['Unnamed: 0'] = pd.to_datetime(df_bom['Unnamed: 0'])
df_bom.set_index('Unnamed: 0', inplace=True)

# Remove duplicated indices in bom
df_bom = df_bom.groupby(df_bom.index).first() # Remove the duplicated index


# Setting df_gt
# df_gt['Unnamed: 0'] = pd.to_datetime(df_gt['Unnamed: 0'])
# df_gt.set_index('Unnamed: 0', inplace=True)

if df_gfs.index[0]>df_bom.index[0]:
    start_date = df_gfs.index[0]
else:
    start_date = df_bom.index[0]

if df_gfs.index[len(df_gfs)-1]<df_bom.index[len(df_bom)-1]:
    end_date = df_gfs.index[len(df_gfs)-1]
else:
    end_date = df_bom.index[len(df_bom)-1]
    
print 'START AND END DATES ARE: ', start_date, end_date
# Chop GFS, BOM and GT from start to end date
df_bom = df_bom.loc[start_date:end_date]
df_gfs = df_gfs.loc[start_date:end_date]

In [ ]:
print 'length of bom and gfs after chopping (May Differ): %d and %d ' %(len(df_bom), len(df_gfs)) 

In [ ]:
# Merge df_gfs and df
df_gfs_bom_combined = df_bom.merge(df_gfs, how='inner', left_index=True, right_index=True)
print len(df_gfs_bom_combined)
df_gfs_bom_combined.dropna(axis=0, how='all', inplace = True) # Drop the rows where all of the elements are nan
print len(df_gfs_bom_combined)



In [ ]:
df_gfs_bom_combined.columns.tolist()

In [ ]:
# Setting bom's selected columns to numeric
df_gfs_bom_combined['Temp_Average'] = pd.to_numeric(df_gfs_bom_combined['Temp_Average'], errors='coerce')
# df_gfs_bom_combined['Station level pressure in hPa'] = pd.to_numeric(df_gfs_bom_combined['Station level pressure in hPa'], errors='coerce')
df_gfs_bom_combined['RelHumidity_Average'] = pd.to_numeric(df_gfs_bom_combined['RelHumidity_Average'], errors='coerce')
df_gfs_bom_combined['rainfall_startend_time'] = pd.to_numeric(df_gfs_bom_combined['rainfall_startend_time'], errors='coerce')

col_list_bom = ['Temp_Average','RelHumidity_Average']

for _col in col_list_bom:
    df_gfs_bom_combined = check_and_fill_null_entries_df_col(df_gfs_bom_combined,_col)

del col_list_bom

# Check Rainfall column
df_gfs_bom_combined = check_and_fill_null_entries_df_col_by_value(df_gfs_bom_combined, 'rainfall_startend_time', 0.0)

# Prepare GT dataframe
df_gt = pd.DataFrame(index=df_gfs_bom_combined.index)
df_gt['rainfall_startend_time'] = df_gfs_bom_combined['rainfall_startend_time'].copy()
# Drop the groundtruth from dataframe
df_gfs_bom_combined.drop('rainfall_startend_time', axis=1, inplace=True)

print 'len of df_gt and df_gfs_bom_combined (MUST Match): %d and %d ' %(len(df_gt), len(df_gfs_bom_combined)) 




In [ ]:
# Scale GFS cols first
df_gfs_bom_combined['pred_temp'] = df_gfs_bom_combined['pred_temp'] - 273.15 # Kelvin to Celsius (BOM unit)
df_gfs_bom_combined['pred_surface_pressure'] = df_gfs_bom_combined['pred_surface_pressure']/100.0 # Pa to Hpa (BOM Unit)

In [ ]:
# plt.scatter( df_gfs_bom_combined['Temp_Average'].values ,df_gfs_bom_combined.pred_temp.values)
# plt.scatter( df_gfs_bom_combined['RelHumidity_Average'].values ,df_gfs_bom_combined.pred_rel_humidity.values)

# df_gfs_bom_combined.Temp_Average.plot()
# df_gfs_bom_combined.pred_temp.plot()

# df_gfs_bom_combined.RelHumidity_Average.plot(figsize=(15,5))
# df_gfs_bom_combined.pred_rel_humidity.plot(figsize=(15,5))


In [ ]:
# # Select those entries that are greater than 10 mm
# outlier_rain_list = df_gt[df_gt.rainfall_t > 10].index.tolist()
# # Remove such entries from the dataset
# print outlier_rain_list
# df_bom.drop(outlier_rain_list,inplace = True)
# df_gfs.drop(outlier_rain_list,inplace = True)
# df_gt.drop(outlier_rain_list,inplace = True)
# print 'outlier entries removed from the dataset\n'
# # Check if GT dataframe is null
# df_gt = check_and_fill_null_entries_df(df_gt)
# print len(df_bom)
# print len(df_gfs)
# print len(df_gt)

In [ ]:
# Pressure is missing in BOM file, add it
# df_bom.Pressure = df_gfs.pred_surface_pressure.values.copy()

In [ ]:
# df_bom.columns.tolist()

In [ ]:
# df_gfs.columns.tolist()

In [ ]:
# # Add vars from the past
# past_hour = 1

# #BOM
# df_bom['temp_p1'] = df_bom['Temperature'].shift(past_hour)
# df_bom['press_p1'] = df_bom['Pressure'].shift(past_hour)
# df_bom['hum_p1'] = df_bom['RelativeHumidity'].shift(past_hour)

# #GFS
# df_gfs['temp_p1'] = df_gfs['pred_temp'].shift(past_hour)
# df_gfs['press_p1'] = df_gfs['pred_surface_pressure'].shift(past_hour)
# df_gfs['hum_p1'] = df_gfs['pred_rel_humidity'].shift(past_hour)

# # Fill nans backwards
# df_bom = check_and_fill_null_entries_df_col_backwards(df_bom, 'temp_p1')
# df_bom = check_and_fill_null_entries_df_col_backwards(df_bom, 'press_p1')
# df_bom = check_and_fill_null_entries_df_col_backwards(df_bom, 'hum_p1')
# df_gfs = check_and_fill_null_entries_df_col_backwards(df_gfs, 'temp_p1')
# df_gfs = check_and_fill_null_entries_df_col_backwards(df_gfs, 'press_p1')
# df_gfs = check_and_fill_null_entries_df_col_backwards(df_gfs, 'hum_p1')

In [ ]:
# df_gfs_bom_combined.columns.tolist()

In [ ]:
# df_gfs_bom_combined.columns.tolist()

In [ ]:
col_list = ['Temp_Average', 'RelHumidity_Average',
           'pred_cloud_cover', 'pred_cloud_cover_bound_cloud_layer', 'pred_convective_cloud',
           'pred_dewp', 'pred_high_tcc', 'pred_low_tcc', 'pred_lw_rad', 'pred_max_wind_press', 'pred_merid_wind',
           'pred_middle_tcc', 'pred_rain_rate', 'pred_rel_humidity', 'pred_sunshine', 'pred_surf_geowind',
           'pred_surface_pressure', 'pred_sw_rad', 'pred_temp', 'pred_total_rain', 'pred_ustorm', 
            'pred_vstorm', 'pred_wind_speed_surf', 'pred_zonal_wind']

for _col in col_list:
    df_gfs_bom_combined = check_and_fill_null_entries_df_col(df_gfs_bom_combined,_col)

df_X = df_gfs_bom_combined[col_list].copy()
df_Y = df_gt.copy()

In [ ]:
# Make sure that df_X is not null
df_X_bool = df_X.isnull().any().any()
print df_X_bool, 'THIS MUST BE FALSE' # This must be FALSE

# df_X.isnull().any()

In [ ]:
# prepare test observations from GFS
# col_list_gfs = ['pred_temp', 'pred_surface_pressure', 'pred_rel_humidity',
#            'temp_p1', 'press_p1', 'hum_p1']
# df_X_test = df_gfs[col_list_gfs].copy()

In [ ]:
# Make sure that df_X is not null
# df_X_bool = df_X_test.isnull().any().any()
# print df_X_bool, 'THIS MUST BE FALSE' # This must be FALSE

In [ ]:
# See if the pressure from BOM and GFS have linear relation among them
# plt.scatter(df_X.Pressure.values,df_X.pred_surface_pressure.values)

In [ ]:
# plt.scatter(df_X['Air Temperature in degrees C'].values,df_X.pred_temp.values)

In [ ]:
# df_X.Pressure.mean()
# print df_X['Mean sea level pressure in hPa'].min(), df_X['Mean sea level pressure in hPa'].max() 
# print df_X['Station level pressure in hPa'].min(), df_X['Station level pressure in hPa'].max() 
print 'min_pressure_gfs: %f max_pressure_gfs=%f\n'%(df_X.pred_surface_pressure.min(), df_X.pred_surface_pressure.max()) 

In [ ]:
# df_X['pred_surface_pressure'].loc[df_X['pred_surface_pressure'] > 1018]

In [ ]:
# If pressure is not having a linear relation with GFS predicted pressure, then replace unaccepted values by mean
# df_X['Station level pressure in hPa'].loc[df_X['Station level pressure in hPa'] < 900] = df_X.Pressure.mean()

In [ ]:
# -----------------------------------------------
#              IF SCALING IS REQUIRED 
# ------------------------------------------------
# Prepare test and train points for scaling
tot_points = len(df_X)
train_points = int(np.floor(train_ratio*tot_points)) 
print 'tot_points: ', tot_points, ' train_points: ', train_points

df_data_matrix = df_X.as_matrix()
print 'Shape of data matrix: ',df_data_matrix.shape

# Generate Train Sequence
x_train = np.copy(df_data_matrix[0:train_points + 1,:])

# Add polynomial features
# poly = PolynomialFeatures(degree=2, interaction_only=False)
# x_train = poly.fit_transform(x_train)

# Normalize data matrix with zero mean and unit covariance
# http://scikit-learn.org/stable/modules/preprocessing.html
scaler = preprocessing.StandardScaler().fit(x_train)



del tot_points
del train_points
del df_data_matrix
del x_train

In [ ]:
df_data_matrix = df_X.as_matrix()
df_gt_rainfall_forecast_matrix = df_Y.as_matrix()

# Adding Polynomial features
# Apply poly transform on df_data_matrix
# df_data_matrix = poly.fit_transform(df_data_matrix)

print 'Shape of data matrix: ',df_data_matrix.shape
print 'Shape of label matrix: ',df_gt_rainfall_forecast_matrix.shape

# Normalize data matrix with zero mean and unit covariance
# http://scikit-learn.org/stable/modules/preprocessing.html
# df_data_matrix = preprocessing.scale(df_data_matrix)
df_data_matrix = scaler.transform(df_data_matrix)


data_matrix = np.copy(df_data_matrix)
label_matrix = np.copy(df_gt_rainfall_forecast_matrix)

print "InputFeedForwardNetwork: ",data_matrix.shape
print "OutputFeedForwardNetwork: ",label_matrix.shape

In [ ]:
# Prepare test and train points
tot_points = data_matrix.shape[0]
train_points = int(np.floor(train_ratio*tot_points)) 
test_points = tot_points - train_points
print 'tot_points: ', tot_points, ' train_points: ', train_points, ' test_points: ', test_points

# Generate Train Sequence
x_train = np.copy(data_matrix[0:train_points + 1,:])
y_train = np.copy(label_matrix[0:train_points + 1,:])
# Generate Test Sequence
x_test = np.copy(data_matrix[train_points + 1:tot_points + 1 ,:])
y_test = np.copy(label_matrix[train_points + 1:tot_points + 1 ,:])

print 'x_train_shape: ', x_train.shape, ' y_train_shape: ', y_train.shape
print 'x_test_shape: ', x_test.shape, ' y_test_shape: ', y_test.shape

In [ ]:
print 'train rainfall in mm: %f' %(np.sum(y_train))
print 'test rainfall in mm: %f' %(np.sum(y_test))

In [ ]:
# plt.plot(df_gt.rainfall_since_last_hour.index, df_gt.rainfall_since_last_hour.values)

In [ ]:
# # Test matrix
# df_test_matrix = df_X_test.as_matrix()
# # Apply poly transform on df_test_matrix
# df_test_matrix = poly.fit_transform(df_test_matrix)

# # Scale it 
# df_test_matrix = scaler.transform(df_test_matrix)

# # Create new variable for test matrix
# testdata_matrix = np.copy(df_test_matrix)

# # Print its shape
# print ' (Must equalize data matrix) Shape of test data matrix: ',testdata_matrix.shape

# # Prepare test and train points
# tot_points = data_matrix.shape[0]
# train_points = data_matrix.shape[0]
# test_points = testdata_matrix.shape[0]
# print 'tot_points: ', tot_points, ' train_points: ', train_points, ' test_points: ', test_points

# # Generate Train Sequence
# x_train = np.copy(data_matrix)
# y_train = np.copy(label_matrix)
# # Generate Test Sequence
# x_test = np.copy(testdata_matrix)
# y_test = np.copy(label_matrix)

# print 'x_train_shape: ', x_train.shape, ' y_train_shape: ', y_train.shape
# print 'x_test_shape: ', x_test.shape, ' y_test_shape: ', y_test.shape

In [ ]:
# create model
model = Sequential()
model.add(Dense(100, input_dim=x_train.shape[1], init='normal', activation='relu')) #1st Hidden Layer
model.add(Dense(200, init='normal', activation='relu')) #2nd Hidden Layer
model.add(Dense(300, init='normal', activation='relu')) #2nd Hidden Layer
model.add(Dense(y_train.shape[1], init='normal')) #output layer
    
# compile model
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# filepath_savemodel = 'C:/Shaukat/code/rainfall/model_stored/feedforward/sensordata/houston/weights.{epoch:02d}-{val_loss:.2f}.h5'
filepath_savemodel = 'C:/Shaukat/code/rainfall/model_stored/feedforward/sensordata/houston/model_v1_best.h5'
# filepath_csvlogger = 'C:/Shaukat/code/rainfall/model_stored/feedforward/sensordata/houston/training_s2.log'

checkpointer = ModelCheckpoint(filepath_savemodel, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
# csv_logger = CSVLogger(filepath_csvlogger, separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0.001)

# history = model.fit(x_train, y_train, nb_epoch=1000, verbose=1, validation_data=(x_test, y_test), batch_size = 32, shuffle=True, callbacks=[checkpointer, csv_logger, reduce_lr])
history = model.fit(x_train, y_train, nb_epoch=100, verbose=1, validation_data=(x_test, y_test), batch_size = 32, shuffle=True, callbacks=[checkpointer, reduce_lr])
# history = model.fit(x_train, y_train, nb_epoch=100, verbose=1, validation_data=(x_test, y_test), batch_size = train_points, shuffle=True, callbacks=[checkpointer])
# history = model.fit(x_train, y_train, nb_epoch=500, verbose=1, validation_data=(x_test, y_test), batch_size = 32, callbacks=[checkpointer])



In [ ]:
model.save('C:/Shaukat/code/rainfall/model_stored/feedforward/sensordata/houston/model_v1_last.h5')

In [ ]:
vec_test = np.copy(y_test)
print vec_test.shape
print 'total test points: ', vec_test.shape[0]

predictions = model.predict(x_test)
pred_vec = np.copy(predictions)
print pred_vec.shape

In [ ]:
# Preprocess block
threshold = 0.01
preproc_y = deepcopy(pred_vec)
preproc_y[preproc_y<=threshold] = 0.0

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
model.evaluate(x_train,y_train)

In [ ]:
# For plot
time_index = np.arange(0,vec_test.shape[0],1)
time_index = time_index.reshape(((len(time_index),1)))

plt_lower_limit = 0
plt_upper_limit = 1000

plt.figure(figsize=(25,8))

# plot

# Full plot
# plt.plot(time_index,vec_test,label="GroundTruth",color='b')
# plt.plot(time_index,pred_vec,label="Prediction",color='g')

# Unpreprocessed Plot
plt.plot(time_index[plt_lower_limit:plt_upper_limit],vec_test[plt_lower_limit:plt_upper_limit],label="GroundTruth",color='b')
plt.plot(time_index[plt_lower_limit:plt_upper_limit],pred_vec[plt_lower_limit:plt_upper_limit],label="Prediction",color='g')

# Preprocessed Plot
# plt.plot(time_index[plt_lower_limit:plt_upper_limit],vec_test[plt_lower_limit:plt_upper_limit],label="GroundTruth",color='b')
# plt.plot(time_index[plt_lower_limit:plt_upper_limit],preproc_y[plt_lower_limit:plt_upper_limit],label="Prediction",color='g')

plt.xlabel('Time')
plt.ylabel('Rainfall prediction in mm')
plt.legend()
plt.title('Groundtruth and Predicted Rainfall per hour')
# img_filename = 'C:\\Users\\ShaukatAbidi\\Documents\\shaukat_python_progs\\learning\\time_series\\feed_forward_nn\\models\\s7_images\\'+str(test_ex)+'.png'
# plt.savefig(img_filename, bbox_inches='tight')
# plt.clf()

In [ ]:
print y_test.shape, predictions.shape

In [ ]:
mean_squared_error(y_test, predictions)

In [ ]:
# saving an array to file
#np.savetxt('predictions.out', predictions, delimiter=',')   # predictions is an array

In [ ]:
# evaluate training 
model.evaluate(x_train,y_train)

In [ ]:
vec_train = np.copy(y_train)
print vec_train.shape
print 'total train points: ', vec_train.shape[0]

pred_on_gt = model.predict(x_train)
pred_on_gt_vec = np.copy(pred_on_gt)
print pred_on_gt_vec.shape

In [ ]:
# For plot
time_index = np.arange(0,vec_train.shape[0],1)
time_index = time_index.reshape(((len(time_index),1)))

plt_lower_limit = 0
plt_upper_limit = 500

plt.figure(figsize=(25,8))

# plot

# Full plot
# plt.plot(time_index,vec_train,label="GroundTruth",color='b')
# plt.plot(time_index,pred_on_gt_vec,label="Prediction",color='g')

# Unpreprocessed Plot
plt.plot(time_index[plt_lower_limit:plt_upper_limit],vec_train[plt_lower_limit:plt_upper_limit],label="GroundTruth",color='b')
plt.plot(time_index[plt_lower_limit:plt_upper_limit],pred_on_gt_vec[plt_lower_limit:plt_upper_limit],label="Prediction",color='g')

# Preprocessed Plot
# plt.plot(time_index[plt_lower_limit:plt_upper_limit],vec_test[plt_lower_limit:plt_upper_limit],label="GroundTruth",color='b')
# plt.plot(time_index[plt_lower_limit:plt_upper_limit],preproc_y[plt_lower_limit:plt_upper_limit],label="Prediction",color='g')

plt.xlabel('Time')
plt.ylabel('Rainfall prediction in mm')
plt.legend()
plt.title('Groundtruth and Predicted Rainfall per hour')
# img_filename = 'C:\\Users\\ShaukatAbidi\\Documents\\shaukat_python_progs\\learning\\time_series\\feed_forward_nn\\models\\s7_images\\'+str(test_ex)+'.png'
# plt.savefig(img_filename, bbox_inches='tight')
# plt.clf()

In [ ]:
plt.plot(y_train)

In [ ]:
plt.plot(y_test)